In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [78]:
p0, p0p, p1, p0pp = sp.symbols('p_0 p_{00} p_1 p_{000}')
a, b, x, g, p, C, E, ell  = sp.symbols('a b x g p C E lambda')

In [189]:
# p_i is the stationary distribution for State i
p0    = p1 / (a*(1-p)*(1-sp.exp(-(1-ell)*(b-C))) + a*p*(1-sp.Min(sp.exp(-(1-ell)*(b-C-E)),1)))
p0p   = p1 * (1-a)
p0pp  = p1 * a
p2    = p1*(a/(1-a))
prest = p1*a/(1-2*a)

In [190]:
eq = sp.Eq(p0+p0p+p0pp+prest+p1, 1)
p1_sym = sp.solve(eq, p1)[0]

In [191]:
p1_sym

a*((2*a*p - 2*a - p + 1)*exp(C*lambda - C - b*lambda + b) + (-2*a*p*Min(1, exp(-C*lambda + C - E*lambda + E + b*lambda - b)) + 2*a + p*Min(1, exp(-C*lambda + C - E*lambda + E + b*lambda - b)) - 1)*exp(2*C*lambda - 2*C - 2*b*lambda + 2*b))*exp(-C*lambda + C + b*lambda - b)/(3*a**2*p - 3*a**2 - 2*a*p + 2*a + (-3*a**2*p*Min(1, exp(-C*lambda + C - E*lambda + E + b*lambda - b)) + 3*a**2 + 2*a*p*Min(1, exp(-C*lambda + C - E*lambda + E + b*lambda - b)) - 1)*exp(C*lambda - C - b*lambda + b))

In [192]:
o_0p    = 1
o_rest = p1*a*(1-a)/(1-2*a)

In [193]:
orphan_full = p0p * o_0p + o_rest
orphan_full

a*p_1*(1 - a)/(1 - 2*a) + p_1*(1 - a)

In [194]:
eq1 = sp.Eq(orphan_full, ell)
eq1

Eq(a*p_1*(1 - a)/(1 - 2*a) + p_1*(1 - a), lambda)

In [195]:
sols = sp.nsolve(eq1.subs({p1:p1_sym}).subs({a:0.3, b:1000.00001, g:0., p:0, C:1., E:0.}), ell, 0.1)

In [196]:
sols

0.201369863013699

In [197]:
# attacker rewards

In [198]:
# 3 cases for f0
f0i_fixed   = C * (a*p*sp.Min(1,sp.exp(-(1-ell)*(b-C-E))) + a*(1-p)*sp.exp(-(1-ell)*(b-C)))
f0ii_fixed  = a*C * (a*p*(1-sp.Min(1,sp.exp(-(1-ell)*(b-C-E)))) + a*(1-p)*(1-sp.exp(-(1-ell)*(b-C))))
f0iii_fixed = (1-a)*(a+g*(1-a))*C * (a*p*(1-sp.Min(1,sp.exp(-(1-ell)*(b-C-E)))) + a*(1-p)*(1-sp.exp(-(1-ell)*(b-C))))
f0iii_fixed

C*(1 - a)*(a + g*(1 - a))*(a*p*(1 - Min(1, exp((lambda - 1)*(-C - E + b)))) + a*(1 - p)*(1 - exp((-C + b)*(lambda - 1))))

In [199]:
# explicit for f0 and f1
f0_fixed   = f0i_fixed + f0ii_fixed + f0iii_fixed
f1_fixed   = C*(a+(1-a)*a)
f0_fixed

C*a*(a*p*(1 - Min(1, exp((lambda - 1)*(-C - E + b)))) + a*(1 - p)*(1 - exp((-C + b)*(lambda - 1)))) + C*(1 - a)*(a + g*(1 - a))*(a*p*(1 - Min(1, exp((lambda - 1)*(-C - E + b)))) + a*(1 - p)*(1 - exp((-C + b)*(lambda - 1)))) + C*(a*p*Min(1, exp((lambda - 1)*(-C - E + b))) + a*(1 - p)*exp((-C + b)*(lambda - 1)))

In [212]:
attack_full_fixed  = p0*f0_fixed + p1*f1_fixed + C*p1*a*(2*a*(1-a)/(1-2*a))
p1_ev = p1_sym.evalf(subs={C:1., E:0.})
attack_final_fixed = attack_full_fixed.evalf(subs={p1:p1_ev, C:1., E:0.})

In [231]:
subs = {a:0.26, b:20000., g:.5, p:0, C:1., E:0.}

In [232]:
sols = sp.nsolve(eq1.subs({p1:p1_sym}).subs(subs), ell, 0.1)
sols

0.178595082789764

In [233]:
attack_final_fixed.subs(subs).subs({ell:sols})*1/(1-sols)

0.264348527237853

In [208]:
attacker_final_fixed.subs(subs)/(attacker_final_fixed.subs(subs)+honest_final_fixed.subs(subs))

0.153504676886140

In [177]:
# p_i is the stationary distribution for State i
p0    = p1 / (a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1)))
p0p   = p1 * (1-a)
p0pp  = p1 * a
p2    = p1*(a/(1-a))
prest = p1*a/(1-2*a)

In [178]:
eq = sp.Eq(p0+p0p+p0pp+prest+p1, 1)
p1_sym = sp.solve(eq, p1)[0]

In [179]:
p1_me = (1/(a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1))) + 1 + (1-a)/(1-2*a))**(-1)
p1_me

1/(1 + 1/(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b))) + (1 - a)/(1 - 2*a))

In [180]:
# confirming the symbolic solver and my by hand solver have same values
p1_sym.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1}), p1_me.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1})

(0.138811963706275, 0.138811963706275)

In [181]:
# 3 cases for f0
f0i_fixed   = C * (a*p*sp.Min(1,sp.exp(-b+C+E)) + a*(1-p)*sp.exp(-b+C))
f0ii_fixed  = a*C * (a*p*(1-sp.Min(1,sp.exp(-b+C+E))) + a*(1-p)*(1-sp.exp(-b+C)))
f0iii_fixed = (1-a)*(a+g*(1-a))*C * (a*p*(1-sp.Min(1,sp.exp(-b+C+E))) + a*(1-p)*(1-sp.exp(-b+C)))
f0iii_fixed

C*(1 - a)*(a + g*(1 - a))*(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b)))

In [182]:
# explicit for f0 and f1
f0_fixed   = f0i_fixed + f0ii_fixed + f0iii_fixed
f1_fixed   = C*(a+(1-a)*a)
f0_fixed

C*a*(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b))) + C*(1 - a)*(a + g*(1 - a))*(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b))) + C*(a*p*Min(1, exp(C + E - b)) + a*(1 - p)*exp(C - b))

In [183]:
attacker_full_fixed  = p0*f0_fixed + p1*f1_fixed + C*p1*a*(2*a*(1-a)/(1-2*a))
p1_ev = p1_sym.evalf(subs={C:1., E:0.})
attacker_final_fixed = sp.simplify(attacker_full_fixed.evalf(subs={p1:p1_ev, C:1., E:0., g:0}))
attacker_final_fixed

a*(0.245252960780962*a**2*(a - 1.0)*(p*(Min(1.0, 2.71828182845905*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) - 0.122626480390481*a*(a - 2.0)*(2.0*a - 1.0)*(p*(Min(1.0, 2.71828182845905*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) - 0.122626480390481*(2.0*a - 1.0)*(1.0*a*(a - 1.0)*(p*(Min(1.0, 2.71828182845905*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) - 1.0*a*(p*(Min(1.0, 2.71828182845905*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) + 1.0*p*exp(b)*Min(1.0, 2.71828182845905*exp(-b)) - 2.71828182845905*p + 2.71828182845905))*(-5.43656365691809*a*p + 5.43656365691809*a + 2.71828182845905*p + (2.0*a*p*Min(1.0, 2.71828182845905*exp(-b)) - 2.0*a - p*Min(1.0, 2.71828182845905*exp(-b)) + 1.0)*exp(b) - 2.71828182845905)/((2.0*a - 1.0)*(p*(Min(1.0, 2.71828182845905*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905))*(-a**2*p + a**2 + 0.666666666666667*a*p - 0.666666666666667*a + 0.1226

In [184]:
g0_fixed   = C*(1-a)
g1_fixed   = C*(1-a)*(1-a)*(1-g)
g0p_fixed  = C*(1-a)
g0pp_fixed = C*(1-a)

In [185]:
honest_full_fixed = p0*g0_fixed + p1*g1_fixed + p0p*g0p_fixed + p0pp*g0pp_fixed
honest_final_fixed = honest_full_fixed.evalf(subs={p1:p1_ev, C:1., E:0., g:0})
honest_final_fixed

1.0*a**2*(1.0 - a)*(-2.0*a*p*Min(1.0, 2.71828182845905*exp(-b)) + 5.43656365691809*a*p*exp(-b) + 2.0*a - 5.43656365691809*a*exp(-b) + p*Min(1.0, 2.71828182845905*exp(-b)) - 2.71828182845905*p*exp(-b) - 1.0 + 2.71828182845905*exp(-b))/(-3.0*a**2*p*Min(1.0, 2.71828182845905*exp(-b)) + 8.15484548537714*a**2*p*exp(-b) + 3.0*a**2 - 8.15484548537714*a**2*exp(-b) + 2.0*a*p*Min(1.0, 2.71828182845905*exp(-b)) - 5.43656365691809*a*p*exp(-b) + 5.43656365691809*a*exp(-b) - 1.0) + 2.0*a*(1.0 - a)**2*(-2.0*a*p*Min(1.0, 2.71828182845905*exp(-b)) + 5.43656365691809*a*p*exp(-b) + 2.0*a - 5.43656365691809*a*exp(-b) + p*Min(1.0, 2.71828182845905*exp(-b)) - 2.71828182845905*p*exp(-b) - 1.0 + 2.71828182845905*exp(-b))/(-3.0*a**2*p*Min(1.0, 2.71828182845905*exp(-b)) + 8.15484548537714*a**2*p*exp(-b) + 3.0*a**2 - 8.15484548537714*a**2*exp(-b) + 2.0*a*p*Min(1.0, 2.71828182845905*exp(-b)) - 5.43656365691809*a*p*exp(-b) + 5.43656365691809*a*exp(-b) - 1.0) + 1.0*a*(1.0 - a)*(-2.0*a*p*Min(1.0, 2.71828182845905*ex

In [186]:
subs = {a:0.33, b:2., g:0., p:0, C:1., E:0.}

In [187]:
attacker_final_fixed.subs(subs)/(attacker_final_fixed.subs(subs)+honest_final_fixed.subs(subs))

0.327951178248070